# Loading Dataset and Importing Libraries

In [2]:
from nltk import sent_tokenize, pos_tag
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.sentiment.util import mark_negation
from string import punctuation
from IPython.display import display
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
import nltk
nltk.download('punkt')
nltk.download('sentiwordnet')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path = '/content/small_corpora.csv'
reviews = pd.read_csv(path)

In [8]:
reviews.head(2)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1.0,False,"03 27, 2015",A35T1X6MXUJ0VO,B00KME8K9C,Josean Rodriguez,This is the worst battlefield yet I think first person shooter is getting old they nee reconfigure the completely it needs mayor overhaul maps but specially the game play some time s you be running around disoriented not knowing what to do too much running around looking for people bad save your money!,This is the worst battlefield yet I think first person shooter is getting ...,1427414400,NaN,{'Format:': ' Video Game'},NaN
1,1.0,True,"02 5, 2015",A2WCWURD1MU5Y,B002BH3N2W,Samantha,I gave this as a gift for Christmas and it was returned to me because it never worked. Disk was scratched upon receipt and seller refuses to replace. BUYER BEWARE!!!!!,BUYER BEWARE,1423094400,NaN,{'Format:': ' Video Game'},NaN


# **Tokenization of sentences and their words**

# 1) Treebank Tokenizer

In [9]:
from nltk.tokenize import TreebankWordTokenizer
from string import punctuation
import string

In [10]:
tb_tokenizer = TreebankWordTokenizer()

In [11]:
reviews["rev_text_lower"] = reviews['reviewText'].apply(lambda rev: str(rev).translate(str.maketrans('', '', punctuation)).replace("<br />", " ").lower())

In [12]:
reviews[['reviewText','rev_text_lower']].sample(2)

,reviewText,rev_text_lower
1729,Not what i Thought !,not what i thought
2667,Fun game!! Better than last year's forsure - so much in depth you can do in career mode.,fun game better than last years forsure so much in depth you can do in career mode


In [13]:
reviews["tb_tokens"] = reviews['rev_text_lower'].apply(lambda rev: tb_tokenizer.tokenize(str(rev)))

In [14]:
pd.set_option('display.max_colwidth', None)
reviews[['reviewText','rev_text_lower','tb_tokens']].sample(3)

reviewText  \
2659                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                Dropship: United Peace Force is a fantastic flight combat game that you should try and pick up cheap because it has been out for awhile now. The whole game is fast, competitive, has a steep learning curve with realistic battle sequences, amazing replay functions (cinematic), huge explosions, 20 tough missions, several Vertical Take-Off & Landing (VTL) vehicles, 10 ground assault vehicles (you also play on the ground sometimes driving tanks and other army vehicles), helicopters, 4 different country landscapes (5 missions per level), a coherent plot, futuristic weapons, sky combat, sea combat, land combat and a solid flight interface.\nThe essence of the game is to drop tanks and vehicles at designated waypoints and then fly around the place targeting the enemy and dog fighting in the sky. Your VTLs range anywhere from full size bombers to light fighter jets for scramble defense. The whole game is set slightly in the future so all the gadgets and planes look really stealthy and sleek.\nThe actual game play itself is amazing with hyper-realistic controls and a very well done environment. Graphics have got better since this was released on the PS2 but still the game is one of the most enjoyable flight combat games out there and is worth every single penny except for one dismal feature!......\nTHE IS NO FREE FLIGHT OR MULTIPLAYER MODE!\nThis brings the game down a star. It would have been simple to do a free flight frag

# 2) Casual Tokenizer

In [15]:
from nltk.tokenize.casual import casual_tokenize

In [16]:
reviews['casual_tokens'] = reviews['rev_text_lower'].apply(lambda rev: casual_tokenize(str(rev)))

In [17]:
reviews[['reviewText','casual_tokens','tb_tokens']].sample(3)

,reviewText,casual_tokens,tb_tokens
2114,"Its a good game for whose want to give some shots and work in team. However, the resident evil essence seems to be gone.\nThere is no terror or survival horror.","[its, a, good, game, for, whose, want, to, give, some, shots, and, work, in, team, however, the, resident, evil, essence, seems, to, be, gone, there, is, no, terror, or, survival, horror]","[its, a, good, game, for, whose, want, to, give, some, shots, and, work, in, team, however, the, resident, evil, essence, seems, to, be, gone, there, is, no, terror, or, survival, horror]"
4470,Great,[great],[great]
4388,"So far, this is the best Spider-Man game.","[so, far, this, is, the, best, spiderman, game]","[so, far, this, is, the, best, spiderman, game]"


# 3) Stemming

In [18]:
from nltk.stem.porter import PorterStemmer

In [19]:
stemmer = PorterStemmer()

In [20]:
reviews['tokens_stemmed'] = reviews['tb_tokens'].apply(lambda words: [stemmer.stem(w) for w in words])

In [21]:
reviews[['tb_tokens','tokens_stemmed']].sample(3)

,tb_tokens,tokens_stemmed
2462,"[its, okay]","[it, okay]"
3484,"[what, an, awesome, product, from, logitech, well, built, and, designed, splendidly, from, the, nice, audible, mouse, clicks, of, its, buttons, to, how, light, the, mouse, actually, is, this, is, a, very, well, thought, out, product, the, mouse, responds, instantaneously, on, wireless, as, far, as, i, can, tell, on, an, older, wireless, logitech, mouse, i, had, to, click, multiple, times, for, the, click, to, register, not, being, attached, to, a, wire, also, means, more, freedom, there, is, no, unnecessary, wire, pull, that, may, occur, with, a, cord, you, can, use, the, mouse, while, tethered, to, reduce, the, response, time, but, wireless, ...]","[what, an, awesom, product, from, logitech, well, built, and, design, splendidli, from, the, nice, audibl, mous, click, of, it, button, to, how, light, the, mous, actual, is, thi, is, a, veri, well, thought, out, product, the, mous, respond, instantan, on, wireless, as, far, as, i, can, tell, on, an, older, wireless, logitech, mous, i, had, to, click, multipl, time, for, the, click, to, regist, not, be, attach, to, a, wire, also, mean, more, freedom, there, is, no, unnecessari, wire, pull, that, may, occur, with, a, cord, you, can, use, the, mous, while, tether, to, reduc, the, respons, time, but, wireless, ...]"
3535,"[state, of, decay, is, an, excellent, game, most, other, zombie, games, are, really, just, zombiethemed, linear, shooters, this, game, is, open, world, with, an, emphasis, on, the, survival, aspect, of, survival, horror, and, it, still, understands, subtlety, which, a, lot, of, developers, seem, to, have, trouble, with, the, melee, and, driving, is, a, little, wonky, but, still, very, satisfying, running, around, scavenging, for, resources, trying, to, keep, morale, high, and, of, course, trying, to, stay, alive, gets, addictive, fast, even, the, music, is, surprisingly, good, giving, this, five, stars, because, while, it, is, not, the, most, polished, zombie, game, out, ...]","[state, of, decay, is, an, excel, game, most, other, zombi, game, are, realli, just, zombiethem, linear, shooter, thi, game, is, open, world, with, an, emphasi, on, the, surviv, aspect, of, surviv, horror, and, it, still, understand, subtleti, which, a, lot, of, develop, seem, to, have, troubl, with, the, mele, and, drive, is, a, littl, wonki, but, still, veri, satisfi, run, around, scaveng, for, resourc, tri, to, keep, moral, high, and, of, cours, tri, to, stay, aliv, get, addict, fast, even, the, music, is, surprisingli, good, give, thi, five, star, becaus, while, it, is, not, the, most, polish, zombi, game, out, ...]"


# Lemmatization

In [22]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [23]:
def penn_to_wn(tag):
    """
        Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [24]:
lemmatizer = WordNetLemmatizer()
def get_lemas(tokens):
    lemmas = []
    for token in tokens:
        pos = penn_to_wn(pos_tag([token])[0][1])
        if pos:
            lemma = lemmatizer.lemmatize(token, pos)
            if lemma:
                lemmas.append(lemma)
    return lemmas

In [25]:
reviews['lemmas'] = reviews['tb_tokens'].apply(lambda tokens: get_lemas(tokens))

In [26]:
reviews[['reviewText','tokens_stemmed','lemmas']].sample(2)

,reviewText,tokens_stemmed,lemmas
3130,"Im in total despair. Why? I cant believe I missed this awesome game for so long! I recently just got my very own PS Vita and was looking for new games to buy. And I kept bypassing this game because of the title and the cover. I assume it was a rhythm game and I definitely was wrong.\n\nDanganRonpa: Trigger Happy Havoc is a mystery visual novel, where each chapter ends with a trial. Think of Ace Attorney meets Hunger Games. The plot is awesome and I just couldn't put the Vita down. It was always one more minute. The game takes place in a dystopian world, where The Tragedy destroy society as we know it. Where rioting and despair rules. But there is one place in the world that houses Hope for the future Hope Peaks Academy. However, what the school didn't realize that the enemy was already inside, turning their shelter into a killing game. Now, you and 14 other students must survive the game. The plot is slowly revealed between 6 chapters. Each chapter lasts about 5-6 hours. Each chapter revolves around 1 case. The main plot is not revealed until the very end, so you don't really know the whole truth throughout the game. So, the game keeps you interested because you become so invested in the characters well-being and figuring out their stories.\n\nNow come to the puzzle and fun part. The trials are the real time stealers here. I have not officially checked the time since I get too engrossed to notice, but the trial themselves takes about 2-3 hours to complete. I like to note I played the game in Kind mode. So, it was not as harsh as mean mode. I found that the trial was not as difficult to figure out as long as you pay attention to the details and figure out the contradiction in the other students argument. For me, I found Ace Attorney trials to be a little bit harder than this game. In regards to the logic, I mean. Its a little more obvious in DanganRonpa. So, I didn't have too much difficulty getting through the trials. I messed up a few times but that was due to the rhythm portion. In any case, if you make a mistake, you can always redo it. The only thing it affects is your final grade, but its not a huge deal. The arguments are timed so you cant spend time thinking it over. Luckily, you can pretty much save at any time except during the prologue.\n\nThe graphics are gorgeous and fit the visual novel idea perfectly. Just imagine flat 2D story book characters against a 3D background. Like a pop-up book. I know Im explaining it badly but the visuals are awesome. As an artist, I love the unique design for each character and it matches their personalities. The backgrounds are clear and detailed.\n\nFor the technical, here is a quick summary. There is no lag. Transition from room to room is pretty seamless. Navigation was easy to use. I only used the English dialogue, so I cant comment on the Japanese. But I usually find the Japanese voice acting to be awesome in any game. Anyway, the English voice acting was good and definitely matches well with each character. There is English subtitles. It's not fully voiced throughout. The trial and cut scenes are completely voiced. At other times, it will be a quick comment from whoever is speaking. I cant comment on translation from the Japanese either. Sorry. There are some extras at the end, but I haven't a chance to explore it yet.\n\nGET THIS GAME! If you like visual novels with puzzle elements, this is for you. And if you are an Ace Attorney fan, get it. However, if you are not reader, then pass this one by. I will like to note there are gruesome murder scenes and foul language for those buying for the children. I have no regrets with this purchase. I have already started DangaRonpa 2 and I will eventually go back to get all the bonuses for this game later. But right now, I got my entertainment worth. This is an awesome game.","[im, in, total, despair, whi, i, cant, believ, i, miss, thi, awesom, game, for, so, long, i, recent, just, got, my, veri, o

# 4) Sentiment Predictor Baseline Model

In [27]:
def get_sentiment_score(tokens):
    score = 0
    tags = pos_tag(tokens)
    for word, tag in tags:
        wn_tag = penn_to_wn(tag)
        if not wn_tag:
            continue
        synsets = wn.synsets(word, pos=wn_tag)
        if not synsets:
            continue

        #most common set:
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())

        score += (swn_synset.pos_score() - swn_synset.neg_score())

    return score

In [28]:
## test
swn.senti_synset(wn.synsets("perfect", wn.ADJ)[0].name()).pos_score()

0.625

In [29]:
reviews['sentiment_score'] = reviews['lemmas'].apply(lambda tokens: get_sentiment_score(tokens))

In [30]:
reviews[['reviewText','lemmas','sentiment_score']].sample(5)

,reviewText,lemmas,sentiment_score
2618,"Picked this up real cheap from Amazon. Great game (VERY GRAPHIC) with a cool cinematic use of the black and white with red. Reminiscent of Sin City in that way. Its the typical attitude of Duke Nukem or the lead Wolfenstein character. Gruff, unforgiving but the 'good guy'. Gets a little repetitive and more environmental traps would have been cool. Camera at times is annoying but by no means ruins the game. Boss battles are easy but fun. Overall game is quick to plow through hence why NOW at this price its awesome.","[picked, up, real, cheap, amazon, great, game, very, graphic, cool, cinematic, use, black, white, red, reminiscent, sin, city, way, typical, attitude, duke, nukem, lead, wolfenstein, character, gruff, unforgiving, good, guy, get, little, repetitive, more, environmental, trap, have, be, cool, camera, time, be, annoy, mean, ruin, game, bos, battle, be, easy, fun, overall, game, be, quick, plow, hence, now, price, awesome]",1.250
3963,Such an awesome addition to the collection. The prince with the princess!,"[such, awesome, addition, collection, prince, princess]",0.625
637,"Ugh this is 1 of the worst games I've ever played in a long time, the puzzles are as irritating as they are repetitive. All you'll be doing throughout the entire game is climbing boxes....yeah.","[be, bad, game, ive, ever, played, long, time, puzzle, be, irritate, be, repetitive, youll, be, do, entire, game, be, climb, boxesyeah]",0.375
1825,the joystick was very worn out and is almost unusable,"[joystick, be, very, worn, be, almost, unusable]",-0.875
3027,"If you love the 9 Line, 45 coin, Video slots with Bonus screens (and or bonus games), this software Video Slots by Masque Publishing is a great set of games. It has 25 Video Slot games; Five of them are Bally Games, that are played in many casinos; and 20 of them are made up games, which are like the games you can play in the casinos. The Bally Slots are Bonus Sevens, Blazing Sevens, Captain jackpot (15 Line-75 coins), Take It Or Leave It and Sunset beach (my personal favorite. The 20 other games include, Astro Bucks, The Great egg game, Green lightning, Java Buzz, llama loot, Money Bees, oil baron, patriot Pride, Polar Payouts, Payout Party, Riches of the East, Riding the Wins, Scatter Bats, Sea Of green, Wealth Potion, reely Reely Rich, Ricochet reels and a few more. By the names of the games you can figure out the premise of each game and how there similar to games in the casinos, and all in all, are great fun!!!","[love, line, coin, video, slot, bonus, screen, bonus, game, software, video, slot, masque, publishing, be, great, set, game, have, video, slot, game, be, bally, game, be, played, many, casino, be, make, up, game, be, game, play, casino, bally, slot, be, bonus, seven, blazing, seven, captain, jackpot, line75, coin, take, leave, sunset, beach, personal, favorite, other, game, include, astro, buck, great, egg, game, green, lightning, java, buzz, llama, loot, money, bee, oil, baron, patriot, pride, polar, payouts, payout, party, rich, east, rid, win, scatter, bat, sea, green, wealth, potion, reely, reely, rich, ricochet, reel, few, more, name, game, figure, premise, game, ...]",1.625


In [31]:
reviews[['reviewText','lemmas','sentiment_score']].sample(5)

,reviewText,lemmas,sentiment_score
1951,"Cartoony look, terrible control, shocking violence, repetitive gameplay. The cutscenes are hand drawn still pictures. The main character -Rau- is difficult to move about and he gets stuck on everything. Collision detection is terrible. And as for the ""innovative"" fight controls, it amounts to little more than button mashing. Halfway through the second ""mission"" -walk along and kill everyone you come across- your sword begins to get stuck in something every time you swing it. Frustrating, dull, and stupid. I quit.","[cartoony, look, terrible, control, shock, violence, repetitive, gameplay, cutscenes, be, hand, drawn, still, picture, main, character, rau, be, difficult, move, get, stuck, everything, collision, detection, be, terrible, innovative, fight, control, amount, little, more, button, mash, halfway, second, mission, walk, kill, everyone, come, sword, begin, get, stuck, something, time, swing, frustrate, dull, stupid, i, quit]",-1.875
3308,"Puyo Pop Fever is the latest installment of the Puyo Puyo puzzler series and has been revamped with an enhanced game engine that includes the new Fever mode, a larger assortment of characters, and additional modes. The Puyo Puyo series has a long and cryptic history here in the States. While a mainstay for the puzzler genre in Japan, Puyo Puyo has been released in many alternate forms, masking its origins over here. Veteran puzzler gamers will recognize the game engine as the same exact one from the Genesis and Game Gear's Dr. Robotnik's Mean Bean Machine and the SNES's Kirby's Avalanche. Fever shares a lot with many other puzzler games; Puyo blocks drop down from the screen, and players try to combine four in a row to send a nuisance Puyo over to their opponent until their screen fills up---you can also unleash chain combos for devastating effects. But the new Fever mode adds a definite frantic dimension to the game. During a match, you build up your Fever gauge by eliminating Puyos, and when it's full, you'll enter Fever mode, which gives you a screen full of triple Puyo sets---one well-placed Puyo can set off a unstoppable chain saving you from the brink of disaster or unleashing a fatal blow to your opponent. The only limitation Fever has is that it supports only two players.\n\nAdditional enhancements in Fever included larger quadruple Puyo clusters, a 16-character lineup (two of which are unlockable), four courses, Free Battle mode in single player, three default rule settings, the ability to edit rules for versus mode, and three Endless Puyo Pop ""classic"" games. While single-player and Endless Puyo Pop modes are engaging, they can only really be considered training for the Versus mode. When you play against an actual person, the gameplay becomes faster paced and furious. Players must learn to adapt and process enormous amounts of information in a split second, paying attention to their current Puyo pile, the next Puyo piece, their Fever gauge, and their nuisance Puyo bank on the top of the screen. If you don't have any triple Puyo sets on your screen, the nuisance Puyo bank will unleash its contents. This will screw up your screen, and you'll hear the annoying chants of your opponent's character as he buries you. The characters are all well balanced and have varying styles and strategies, and they seem to be well designed to annoy the heck out of you while you play. There's nothing like the feeling of doom as you're fixated on your screen while you here the chants of the opposing character making chain after chain combos. With single, versus, and three endless modes, and 16 available characters, Fever is as addictive as it gets.","[puyo, pop, fever, be, late, installment, puyo, puyo, puzzler, series, have, be, revamped, enhance, game, engine, include, new, fever, mode, large, assortment, character, additional, mode, puyo, puyo, series, have, long, cryptic, history, here, state, mainstay, puzzler, genre, japan, puyo, puyo, have, be, r